# Import Lib

In [1]:
import open3d as o3d
import numpy as np

import pandas
from sklearn.decomposition import PCA

import copy

# Read scan model

## import obj file

In [2]:
scan_obj = o3d.io.read_triangle_mesh("./data/femur_half_4.obj")
print(scan_obj)

geometry::TriangleMesh with 35027 points and 68786 triangles.


In [3]:
o3d.visualization.draw_geometries([scan_obj], mesh_show_wireframe=True)

# Scale unit length to 1mm (coordinate 1000x)

## point center

In [4]:
points_center = scan_obj.get_center()
print(points_center)

[ 0.24559977 -0.49631218 -0.23152044]


In [5]:
scan_obj.scale(1000.0, points_center)

geometry::TriangleMesh with 35027 points and 68786 triangles.

## save file

In [6]:
o3d.io.write_triangle_mesh("./data/femur_half_4_scaled.obj", scan_obj)

True

# Delete floor plane

## change to point cloud

In [7]:
number_of_points = np.asarray(scan_obj.vertices).shape[0]

In [8]:
scan_obj.compute_vertex_normals()
scan_pcd = scan_obj.sample_points_uniformly(number_of_points)
# o3d.visualization.draw_geometries([scan_pcd])

## find plane using RANSAC

### plane function: ax + by + cz + d = 0

In [9]:
plane_model, inliers = scan_pcd.segment_plane(distance_threshold=2,
                                              ransac_n=3,
                                              num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: 0.00x + 1.00y + -0.01z + 6.81 = 0


In [10]:
# floor
inlier_cloud = scan_pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

# bone 
bone_cloud = scan_pcd.select_by_index(inliers, invert=True)

# o3d.visualization.draw_geometries([inlier_cloud, bone_cloud])

# Delete outliers

In [11]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [12]:
cl, ind = bone_cloud.remove_statistical_outlier(nb_neighbors=10,
                                                std_ratio=2.0)
# display outliers
display_inlier_outlier(bone_cloud, ind)

Showing outliers (red) and inliers (gray): 


In [13]:
bone_cloud = bone_cloud.select_by_index(ind)
# o3d.visualization.draw_geometries([bone_cloud])

### write point cloud to file

In [14]:
# o3d.io.write_point_cloud("./data/femur_half_4_bone_point_cloud.ply", bone_cloud)

# Read Target model

## read mesh from target model

In [15]:
target_obj = o3d.io.read_triangle_mesh("./data/Femur.stl")
print(target_obj)

geometry::TriangleMesh with 10415430 points and 3471810 triangles.


### down sample to point clould

In [16]:
number_of_points = np.asarray(bone_cloud.points).shape[0]
print(number_of_points)

5964


In [17]:
target_pcd = target_obj.sample_points_uniformly(number_of_points*3, seed=10)
# o3d.visualization.draw_geometries([target_pcd], mesh_show_wireframe=True)

In [18]:
# o3d.io.write_point_cloud("./data/femur_target_point_cloud.ply", target_pcd)

# Calculate PCA

## Target

### subtract mean each column

In [19]:
target_bone_points_cloud_array = np.asarray(target_pcd.points)

In [20]:
target_bone_points_cloud_array = target_bone_points_cloud_array - target_bone_points_cloud_array.mean(axis=0, keepdims=True)
print(target_bone_points_cloud_array)

[[ -90.55469037   22.83530695 -180.37680893]
 [ -90.84212097   25.44519751 -180.31737287]
 [ -91.27618579   25.31480551 -180.19959771]
 ...
 [  84.01156346  -17.75767594  204.09502917]
 [  85.8306403   -14.46792821  204.10202374]
 [  82.18522422  -14.62224043  204.11853827]]


### PCA

In [21]:
target_pca = PCA(n_components=3)
target_pca.fit(target_bone_points_cloud_array)
print(target_pca.explained_variance_ratio_)

[0.97502814 0.01584185 0.00913001]


In [22]:
target_bone_points_cloud_array = target_pca.transform(target_bone_points_cloud_array)

In [23]:
target_bone_pca_pcd = o3d.geometry.PointCloud()
target_bone_pca_pcd.points = o3d.utility.Vector3dVector(target_bone_points_cloud_array)

In [24]:
# o3d.visualization.draw_geometries([target_bone_pca_pcd], mesh_show_wireframe=True)

## Scan

## subtract mean each column

In [25]:
scan_bone_points_cloud_array = np.asarray(bone_cloud.points)

In [26]:
scan_bone_points_cloud_array = scan_bone_points_cloud_array - scan_bone_points_cloud_array.mean(axis=0, keepdims=True)
print(scan_bone_points_cloud_array)

[[ 195.6493701     5.36482974   72.77265074]
 [ 200.64326472    6.6587375    72.97136928]
 [ 201.61424879    4.65614549   73.42438614]
 ...
 [-152.80220675  -20.97813386  -38.80940997]
 [-155.8017711   -20.51966519  -41.43718035]
 [-180.54500712  -25.8044468   -47.72561581]]


### PCA

In [27]:
scan_pca = PCA(n_components=3)
scan_pca.fit(scan_bone_points_cloud_array)
print(scan_pca.explained_variance_ratio_)

[0.97321351 0.01824985 0.00853664]


In [28]:
scan_bone_points_cloud_array = scan_pca.transform(scan_bone_points_cloud_array)

In [29]:
scan_bone_pca_pcd = o3d.geometry.PointCloud()
scan_bone_pca_pcd.points = o3d.utility.Vector3dVector(scan_bone_points_cloud_array)

In [30]:
o3d.visualization.draw_geometries([scan_bone_pca_pcd], mesh_show_wireframe=True)

## visualize together

In [31]:
target_bone_pca_pcd.paint_uniform_color([1, 0, 0])
scan_bone_pca_pcd.paint_uniform_color([0, 1, 0])

geometry::PointCloud with 5964 points.

In [32]:
o3d.io.write_point_cloud("./data/target_temp.ply", target_bone_pca_pcd)
o3d.io.write_point_cloud("./data/scan_temp.ply", scan_bone_pca_pcd)

True

In [33]:
o3d.visualization.draw_geometries([target_bone_pca_pcd, scan_bone_pca_pcd], mesh_show_wireframe=True)

# Registration

## define source and target, helper plot fucntion

In [34]:
source = scan_bone_pca_pcd
target = target_bone_pca_pcd

In [35]:
def draw_registration_result(source, target, transformation, name):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([0, 1, 0])
    target_temp.paint_uniform_color([1, 0, 0])
    source_temp.transform(transformation)
    
    # add aixs
    aix_points = [[0, 0, 0],
                  [0, 0, 100],
                  [100, 0, 0],
                  [0, 100, 0]]
    aix_lines = [[0, 1],
                 [0, 2],
                 [0, 3]]
    colors = [[0, 0, 0] for i in range(len(aix_lines))]
    aix_line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(aix_points),
        lines=o3d.utility.Vector2iVector(aix_lines),
    )
    aix_line_set.colors = o3d.utility.Vector3dVector(colors)
    
    o3d.visualization.draw_geometries([source_temp, target_temp, aix_line_set], name)

## initial transformation matrix

In [36]:
source_array = np.asarray(source.points)
target_array = np.asarray(target.points)

In [37]:
source_max_in_columns = np.amax(source_array, axis=0)
target_max_in_columns = np.amax(target_array, axis=0)

source_min_in_columns = np.amin(source_array, axis=0)
target_min_in_columns = np.amin(target_array, axis=0)

In [38]:
print('source:')
print(source_min_in_columns)
print(source_max_in_columns)

print('target: ')
print(target_min_in_columns)
print(target_max_in_columns)

source:
[-206.49121416  -40.21611976  -29.56818195]
[235.0414499   46.09082496  34.54498367]
target: 
[-204.06291084  -34.47748846  -24.56640372]
[221.16770043  48.68529274  38.02743083]


In [39]:
strech = np.divide(target_max_in_columns - target_min_in_columns, source_max_in_columns - source_min_in_columns) 
print(strech)

[0.96307849 0.96356998 0.97630235]


In [40]:
threshold =8
trans_init = np.asarray([[strech[0], 0, 0, 0],
                         [0, strech[1], 0, 0],
                         [0, 0, strech[2], 0],
                         [0, 0,  0, 1]])
draw_registration_result(source, target, trans_init, 'init matrix result')

In [41]:
evaluation = o3d.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)

registration::RegistrationResult with fitness=9.431590e-01, inlier_rmse=4.004533e+00, and correspondence_set size of 5625
Access transformation to get result.


In [42]:
reg_p2p = o3d.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.registration.TransformationEstimationPointToPoint(),
        o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)

registration::RegistrationResult with fitness=9.931254e-01, inlier_rmse=3.168534e+00, and correspondence_set size of 5923
Access transformation to get result.
Transformation is:
[[ 9.63075242e-01  2.20966733e-03 -1.19019941e-03 -7.32660888e+00]
 [-2.12346309e-03  9.61330853e-01  6.64840287e-02  1.08286064e-01]
 [ 1.32174674e-03 -6.56141710e-02  9.74035289e-01  2.36719423e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [43]:
# draw_registration_result(source, target, reg_p2p.transformation, 'p2p')
# compute cpd registration
result = copy.deepcopy(source)
result = result.transform(reg_p2p.transformation)

# draw result: source: green, target: red, registed: blue
source.paint_uniform_color([0, 1, 0])
target.paint_uniform_color([1, 0, 0])
result.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([source, target, result])

In [44]:
o3d.io.write_point_cloud("./data/result_temp.ply", result)

True

# -------------------------------------------------

In [45]:
# from probreg import cpd

In [46]:
# tf_param, _, _ = cpd.registration_cpd(source, target)

In [47]:
# # compute cpd registration
# result = copy.deepcopy(source)
# result.points = tf_param.transform(result.points)

# # draw result
# source.paint_uniform_color([0, 1, 0])
# target.paint_uniform_color([1, 0, 0])
# result.paint_uniform_color([0, 0, 1])
# o3d.visualization.draw_geometries([source, target, result])